# Inundation Maps

In [ ]:
import os
import sys
from osgeo import gdal
from osgeo import ogr
#import geodatasets
import qgis
from qgis.gui import *
from qgis.core import *
from qgis.utils import plugins
from PyQt5.QtCore import *
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as cx
import random

sys.path.append('/usr/share/qgis/python/plugins/processing')
sys.path.append('/usr/share/qgis/python/plugins/')
sys.path.append('/usr/share/qgis/python/')
#sys.path.append('/usr/share/miniconda3/envs/qgis/Library/python/plugins')

QgsApplication.setPrefixPath('/usr', True)
app = QgsApplication([], False)
app.initQgis()
from qgis.analysis import QgsNativeAlgorithms, QgsRasterCalculator, QgsRasterCalculatorEntry
import processing
from processing.core.Processing import Processing
Processing.initialize()

# Add your own data

In [ ]:
#For better plotting results order the runup values ASCENDING
runup_values= [3,7,10]

In [ ]:
uri_digital_terrain_model = r"C:\Users\Bea\Documents\NEAM_commitment\tsunami_inundation_maps_training\syracuse_bay\syracuse_bay_32633_clipped.tif"
digital_terrain_model=QgsRasterLayer(uri_digital_terrain_model, "DEM")

In [ ]:
uri_shoreline = r"C:\Users\Bea\Documents\NEAM_commitment\tsunami_inundation_maps_training\syracuse_bay\sr_shoreline_polygon_32633.shp"
shoreline = QgsVectorLayer(uri_shoreline, "shoreline", "ogr")


In [ ]:
output_folder=r"C:\Users\Bea\Documents\NEAM_commitment\tsunami_inundation_maps_training\syracuse_bay\output_JN"

In [ ]:
uri_river_network = r"C:\Users\Bea\Documents\NEAM_commitment\tsunami_inundation_maps_training\syracuse_bay\sr_rivers_32633.shp"
river_network=QgsVectorLayer(uri_river_network, "river_network", "ogr")

In [ ]:
uri_lakes = r"C:\Users\Bea\Documents\NEAM_commitment\tsunami_inundation_maps_training\syracuse_bay\sr_lakes_32633.shp"
lakes=QgsVectorLayer(uri_lakes, "lakes", "ogr")


## Map calculation and Plotting

In [ ]:
r = lambda: random.randint(0,255)
shapefile_dict = {} 

for i in range (0, len(runup_values)):
    runup_value=runup_values[i]
    runup_value_name=str(runup_value)
    runup_text_for_filename=os.path.join(output_folder,runup_value_name+"m.shp")
    wet_dry_area_vector=os.path.join(output_folder,"wd_vector_"+runup_value_name+".shp")
    wet_dry_area_raster=os.path.join(output_folder,"wd_raster_"+runup_value_name+".tif")
    wet_area_vector=os.path.join(output_folder,"wet_area_vector_"+runup_value_name+".shp")

    
    saving_path_wetdry=os.path.join(output_folder,"wet_dry_area_vector_"+runup_value_name+"m.shp")
    saving_path_wetdry_raster=os.path.join(output_folder,"wet_dry_area_raster_"+runup_value_name+"m.tif")
    saving_path_wet=os.path.join(output_folder,"wet_area_vector_"+runup_value_name+"m.shp")
    print("Runup value: ", runup_value, " m")
    

###### 1. Buffer per la costa in base al moltiplicatore di runup

    if not shoreline.isValid():
        print("Layer failed to load!")
    else:
        buffer_distance = 200 * runup_value
        alg_params = {
                    'DISSOLVE': True,
                    'DISTANCE': buffer_distance,
                    'INPUT': shoreline,
                    'END_CAP_STYLE': 0,
                    'JOIN_STYLE': 0,
                    'MITER_LIMIT': 2,
                    'SEGMENTS': 5,
                    'SEPARATE_DISJOINT': False,
                    'OUTPUT': QgsProcessing.TEMPORARY_OUTPUT
                }
    
        feedback = QgsProcessingFeedback()
        Buffer_coast=processing.run("native:buffer", alg_params, feedback=feedback)
    
###### 2. Buffer per i laghi
    buffer_distance = 200 * runup_value
    if not lakes.isValid():
        print("Layer failed to load!")
    else:
        alg_params = {
                    'DISSOLVE': True,
                    'DISTANCE': buffer_distance,
                    'INPUT': lakes,
                    'END_CAP_STYLE': 0,
                    'JOIN_STYLE': 0,
                    'MITER_LIMIT': 2,
                    'SEGMENTS': 5,
                    'SEPARATE_DISJOINT': False,
                    'OUTPUT': QgsProcessing.TEMPORARY_OUTPUT
                }
        feedback = QgsProcessingFeedback()
        Buffer_lakes=processing.run("native:buffer", alg_params, feedback=feedback)
    
###### 3. Buffer per la costa da usare per ritagliare i fiumi

    if not shoreline.isValid():
        print("Layer failed to load!")
    else:
        buffer_distance = 400 * runup_value
        alg_params = {
                    'DISSOLVE': True,
                    'DISTANCE': buffer_distance,
                    'INPUT': shoreline,
                    'END_CAP_STYLE': 0,
                    'JOIN_STYLE': 0,
                    'MITER_LIMIT': 2,
                    'SEGMENTS': 5,
                    'SEPARATE_DISJOINT': False,
                    'OUTPUT': QgsProcessing.TEMPORARY_OUTPUT
                }
        feedback = QgsProcessingFeedback()
        Bf_coastToClipRivers=processing.run("native:buffer", alg_params, feedback=feedback)

##### # 4. Ritaglia la rete fluviale usando il buffer dalla costa (se la rete fluviale è fornita)

    if not river_network.isValid():
        print("Layer failed to load!")
    else:
        alg_params = {
                    'INPUT':river_network,
                    'OVERLAY': Bf_coastToClipRivers['OUTPUT'],
                    'OUTPUT': QgsProcessing.TEMPORARY_OUTPUT
                }
        feedback = QgsProcessingFeedback()
        Clip_riversByBfCoast=processing.run("native:clip", alg_params, feedback=feedback)
    
    
##### # 5. Buffer per i fiumi ritagliati (se esistono)
    if not Clip_riversByBfCoast['OUTPUT'].isValid():
        print("Layer failed to load!")
    else:
        buffer_distance = runup_value * 100
        alg_params = {
                    'DISSOLVE': True,
                    'DISTANCE': buffer_distance,
                    'INPUT': Clip_riversByBfCoast['OUTPUT'],
                    'END_CAP_STYLE': 0,
                    'JOIN_STYLE': 0,
                    'MITER_LIMIT': 2,
                    'SEGMENTS': 5,
                    'SEPARATE_DISJOINT': False,
                    'OUTPUT': QgsProcessing.TEMPORARY_OUTPUT
                }
        feedback = QgsProcessingFeedback()
        Buffer_rivers=processing.run("native:buffer", alg_params, feedback=feedback)

###### 6. Unione dei buffer (solo se esistono entrambi)
    layers_to_merge = []
    if Buffer_coast['OUTPUT'].isValid():
        layers_to_merge.append(Buffer_coast['OUTPUT'])
    if Buffer_rivers['OUTPUT'].isValid():
        layers_to_merge.append(Buffer_rivers['OUTPUT'])
    if Buffer_lakes['OUTPUT'].isValid:
        layers_to_merge.append(Buffer_lakes['OUTPUT'])
    if len(layers_to_merge)>0:
        alg_params = {
                    'LAYERS': layers_to_merge,
                    'CRS': None,
                    'OUTPUT': QgsProcessing.TEMPORARY_OUTPUT
                }
        Merge_buffers = processing.run('native:mergevectorlayers', alg_params, feedback=feedback)
    
####### 7. Dissolvi i buffer uniti

    if not Merge_buffers['OUTPUT'].isValid():
        print("")
    else:
        alg_params = {
                    'FIELD': [],
                    'INPUT': Merge_buffers['OUTPUT'],
                    'SEPARATE_DISJOINT': False,
                    'OUTPUT': QgsProcessing.TEMPORARY_OUTPUT
                }
        Dissolve_merged_bf = processing.run('native:dissolve', alg_params, feedback=feedback)

######## 8. Ritaglia il raster usando il buffer dissolto
        if not Merge_buffers['OUTPUT'].isValid():
            print("")
        else:
            alg_params = {
                    'ALPHA_BAND': False,
                    'CROP_TO_CUTLINE': True,
                    'DATA_TYPE': 0,
                    'INPUT': digital_terrain_model,
                    'MASK': Dissolve_merged_bf['OUTPUT'],
                    'MULTITHREADING': False,
                    'NODATA': None,
                    'OUTPUT': QgsProcessing.TEMPORARY_OUTPUT
                }
            ClipRasterByMaskLayer = processing.run('gdal:cliprasterbymasklayer', alg_params,  feedback=feedback, is_child_algorithm=True)

########9. Calcola l'area wet/dry
        if not QgsRasterLayer(ClipRasterByMaskLayer['OUTPUT']).isValid():
            print("")
        else:
            saving_path_wetdry_raster=os.path.join(output_folder,"wet_and_dry_area"+runup_value_name+"m.tif")
            alg_params = {
                    'CELL_SIZE': None,
                    'CRS': 'ProjectCrs',
                    'EXPRESSION': f'"A@1" <= {runup_value}',
                    'EXTENT': None,
                    'LAYERS': QgsRasterLayer(ClipRasterByMaskLayer['OUTPUT']),
                    'OUTPUT':  saving_path_wetdry_raster
                }
            Calc_wet_dry = processing.run('native:modelerrastercalc', alg_params, feedback=feedback, is_child_algorithm=True)

########10. Polygonizza il raster (converti in vettore)
    if not QgsRasterLayer(Calc_wet_dry['OUTPUT']).isValid():
            print("")
    else:
            saving_path_wetdry=os.path.join(output_folder,"area_wetdry_"+runup_value_name+"m.shp")
            alg_params = {
                    'BAND': 1,
                    'EIGHT_CONNECTEDNESS': True,
                    'EXTRA': None,
                    'FIELD': 'wet_dry',
                    'INPUT': QgsRasterLayer(Calc_wet_dry['OUTPUT']),
                    'OUTPUT':  saving_path_wetdry
                }
            PolygonizeRasterToVector = processing.run('gdal:polygonize', alg_params, feedback=feedback)

########11. Filtra solo le aree bagnate (wet_dry = 1)
    if not QgsVectorLayer(PolygonizeRasterToVector['OUTPUT']).isValid():
            print("")
    else:
            saving_path_wet=os.path.join(output_folder,"wet_area_"+runup_value_name+"m.shp")
            alg_params = {
                    'INPUT': QgsVectorLayer(PolygonizeRasterToVector['OUTPUT']),
                    'EXPRESSION': '"wet_dry" = 1',
                    'OUTPUT':  saving_path_wet
                }
            FilteredWetArea = processing.run('native:extractbyexpression', alg_params, feedback=feedback)
            #create the plotting variables and read the shapefiles
            name_gdf="'gdf"+(str(i))+"'"
            shapefile_dict.update({ str(name_gdf):gpd.read_file(os.path.join(output_folder,"wet_area_"+runup_value_name+"m.shp"))})
########12. Print the map
# Read the coastline
gdf_coastline = gpd.read_file(uri_shoreline)

# Create a plot with a figure and an axes object
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

# Plot each shapefile on the same axes
for key, value in reversed(shapefile_dict.items()):
    (shapefile_dict[key]).plot(ax=ax, color=('#%02X%02X%02X' % (r(),r(),r())), alpha=0.7, label='Layer')
gdf_coastline.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=1.5)

# Add the base map from contextily
cx.add_basemap(ax, crs=gdf_coastline.crs.to_string(),source=cx.providers.CartoDB.Voyager)

# Add a legend and title
ax.set_title('Inundation Map')

# Show the plot
plt.show()

# Stop QGIS appllication
app.exitQgis()
app.exit()